In [1]:
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv


## Display all rows of pandas dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
'''
function name: fix_column_names

purpose: Fixing the column names, making them smaller, informative, and consistent

input: The raw counts dataframe for either genes or transcripts 

output: Same dataframe with improved column names
'''

def fix_column_names(df, is_gene=False):
    
    ## Check if this is a gene counts object
    if is_gene:
        
        ## Get count column names and create list of new column names
        df["gene_id"] = df[df.columns[0]]
        df.drop(columns=df.columns[0], inplace=True)
        
        ## gene_id comes in as index for gene counts data, make it into the first column instead
        cols = list(df.columns)
        cols = [cols[-1]] + cols[:-1]
        df = df[cols]
        df.reset_index(inplace=True, drop=True)
        
        ## Define counts columns and initiate new_columns list
        count_columns = df.columns[1:].tolist()
        list_new_names = ["gene_id"]
    
    ## If it is a transcript dataset
    else:
        ## Set count columns and create list of new names
        count_columns = df.columns[2:].tolist()
        list_new_names = ["transcript_id", "gene_id"]
    
    ## Fix names one by one and add to list of new names
    for col in count_columns:
        col = col.split("_mapped")[0] + "_counts"
        list_new_names.append(col)
    
    ## Rename columns
    df.columns = list_new_names
    
    return df 

In [3]:
'''
function name: parse_df_columns

purpose: parsing the last aggregate column of the gtf/gff3 into useful columns and cleaning non-relevant columns

input: dataframe containining "raw" gtf/gff

output: dataframe containing gtf with useful columns ["gene_id", "transcript_id", etc...]
'''

def parse_df_columns(df, is_ref=True, is_transcript=False, is_exon=False):

    if is_ref:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)
        
        ## Get gene names
        df["gene_name"] = df["other"].str.split("gene_name \"", expand=True)[1].str.split('\";', expand=True)[0]
        
        ## Get get transcript biotype
        df["gene_biotype"] = df["other"].str.split('gene_biotype "', expand=True)[1].str.split('"', expand=True)[0]
        
        if is_transcript:
            df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]
            df["transcript_biotype"] = df["other"].str.split('transcript_biotype "', expand=True)[1].str.split('"', expand=True)[0]
        
        if is_exon:
            df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]
            df["transcript_biotype"] = df["other"].str.split('transcript_biotype "', expand=True)[1].str.split('"', expand=True)[0]
            df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

            
        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)
        

    else:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)

        ## Get transcript ids
        df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]

        ## Get exon number
        df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)

    for col in df.columns:
        df.loc[df[col].isnull(), col] = np.NaN
        

    return df

In [4]:
'''
function name: calculate_cpm

purpose: Calculate CPM for the each sample given

input: Counts dataset

output: Counts dataset with CPM columns as well
'''

def calculate_cpm(df, is_gene=False):

    ## Set count columns if dataframe is gene counts
    if is_gene:
        count_columns = df.columns[1:].tolist()
    
    ## Set count columns if dataframe is transcript counts
    else:
        count_columns = df.columns[2:].tolist()

    ## Loop through counts columns to calculate CPM and add to the dataframe
    for col in count_columns:
        
        df[col] = df[col]
        cpm_name = col.replace("_counts", "_CPM")
        df[cpm_name] = ((df[col]/(df[col].sum())) * 1000000)
    
    return df  

In [5]:
## Import Data

## Open original reference
original_ref = pd.read_csv("../../../../references/bernardo/Homo_sapiens.GRCh38.107_ERCC.gtf", header=None, delimiter="\t", low_memory=False, 
                       names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"], comment="#")



## Bambu reference with novel and annotated transcripts
bambu_ref = pd.read_csv("../../../../data/bernardo/raw/uky_aged_stringent/bambu_discovery/extended_annotations.gtf", header=None, delimiter="\t",
                        low_memory=False, names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])

## New Transcript Events
df_events = pd.read_csv("../../../../data/bernardo/processed/03.gene_and_transcripts_descriptive_stats/uky_aged_stringent_novel_events.tsv", sep="\t")


## Bambu counts matrix
df = pd.read_csv("../../../../data/bernardo/raw/uky_aged_stringent/bambu_discovery/counts_transcript.txt", 
                           delimiter="\t", low_memory=False, header=0)

In [6]:
## Get all transcript IDs from ENSEMBL 94 GTF

df_ensembl_94 = pd.read_csv("../../../../references/bernardo/Homo_sapiens.GRCh38.94.gtf", header=None, delimiter="\t",
                        low_memory=False, names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])

df_ensembl_94 = df_ensembl_94.loc[df_ensembl_94["type"] == "transcript"].copy()

df_ensembl_94 = parse_df_columns(df_ensembl_94, is_ref=True, is_transcript=True)

ensembl_94_transcript_ids = df_ensembl_94["transcript_id"].copy()

In [7]:
## Parse through original reference to get all needed info

orig_ref_transcripts = original_ref.loc[original_ref["type"]=="transcript"].copy()
orig_ref_exons = original_ref.loc[((original_ref["type"]=="exon") | (original_ref["type"]=="CDS"))].copy()

orig_ref_transcripts = parse_df_columns(orig_ref_transcripts, is_ref=True, is_transcript=True)
orig_ref_exons = parse_df_columns(orig_ref_exons, is_ref=True, is_exon=True)

orig_ref_transcripts.drop(columns=["source"], inplace=True)
orig_ref_transcripts["exon_number"] = np.nan

orig_ref_exons.drop(columns=["source"], inplace=True)

orig_ref = pd.concat([orig_ref_transcripts, orig_ref_exons]).sort_values(by=["chr", "start", "type", "end"], 
                                                                                    ascending=[True, True, False, True])


orig_ref.loc[orig_ref["transcript_id"].isin(ensembl_94_transcript_ids), "annotation_status"] = "Annotated in ENSEMBL 94 (2019)"
orig_ref.loc[~orig_ref["transcript_id"].isin(ensembl_94_transcript_ids), "annotation_status"] = "NOT Annotated in ENSEMBL 94 (2019)"

orig_ref["discovery_category"] = "Annotated"

In [8]:
## Cleanup df_events
df_events = df_events[["TXNAME", "txClassDescription"]]

## Cleanup net transcript classes
df_events.loc[df_events["txClassDescription"] == "newGene-spliced", "txClassDescription"] = "New Gene Body"

df_events.loc[df_events["txClassDescription"] == "newWithin", "txClassDescription"] = "New Combo of Known Junctions/Exons"
df_events.loc[df_events["txClassDescription"] == "newFirstJunctionnewFirstExon", "txClassDescription"] = "New First Exon"
df_events.loc[df_events["txClassDescription"] == "newLastJunctionnewLastExon", "txClassDescription"] = "New Last Exon"
df_events.loc[df_events["txClassDescription"] == "newFirstJunction", "txClassDescription"] = "New Last Exon"
df_events.loc[df_events["txClassDescription"] == "allNew", "txClassDescription"] = "All New"

df_events.loc[df_events["txClassDescription"] == "newFirstJunction", "txClassDescription"] = "New Junction"
df_events.loc[df_events["txClassDescription"] == "newLastJunctionnewJunction", "txClassDescription"] = "New Junction"
df_events.loc[df_events["txClassDescription"] == "newFirstJunctionnewJunction", "txClassDescription"] = "New Junction"
df_events.loc[df_events["txClassDescription"] == "newLastJunction", "txClassDescription"] = "New Junction"
df_events.loc[df_events["txClassDescription"] == "newJunction", "txClassDescription"] = "New Junction"

df_events.loc[df_events["txClassDescription"] == "newLastJunctionnewJunctionnewLastExon", "txClassDescription"] = "New Last Exon & New Junction"
df_events.loc[df_events["txClassDescription"] == "newFirstJunctionnewJunctionnewFirstExon", "txClassDescription"] = "New First Exon & New Junction"
df_events.loc[df_events["txClassDescription"] == "newLastJunctionnewFirstJunctionnewJunctionnewFirstExonnewLastExon", "txClassDescription"] = "New First & Last Exon & New Junction"
df_events.loc[df_events["txClassDescription"] == "newLastJunctionnewFirstJunctionnewJunctionnewFirstExon", "txClassDescription"] = "New First Exon & New Junction"

## Change column names
df_events.columns = ["transcript_id", "discovery_category"]

In [9]:
## Parse through bambu reference
bambu_ref = parse_df_columns(bambu_ref, is_ref=False)

## Only keep novel genes and Transcripts
bambu_ref = bambu_ref.loc[bambu_ref["transcript_id"].str.startswith("BambuTx")]

## Create discovery category column
bambu_ref["annotation_status"] = "unnanotated (newly discovered)"
bambu_ref["transcript_biotype"] = "unnanotated (newly discovered)"
bambu_ref = bambu_ref.merge(df_events, on="transcript_id", how="inner").drop_duplicates()

## Get gene names
gene_names = original_ref.loc[original_ref["type"]=="gene"].copy()
gene_names = parse_df_columns(gene_names, is_ref=True)
gene_names = gene_names[["gene_id", "gene_name"]]

## Add gene_name
bambu_ref = bambu_ref.merge(gene_names, on="gene_id", how="left").drop_duplicates().drop(columns="source")

In [10]:
## Final Reference
final_ref = pd.concat([orig_ref, bambu_ref]).sort_values(by=["chr", "start", "type", "end"], 
                                                                                    ascending=[True, True, False, True])

In [11]:
### Fix transcript biotypes

## Create CDS_not_defined classification
final_ref.loc[((final_ref["gene_biotype"] == "protein_coding") & (final_ref["transcript_biotype"] == "processed_transcript")), "transcript_biotype"] = "cds_not_defined"

## Create other classification
final_ref.loc[~final_ref["transcript_biotype"].isin(["protein_coding", "nonsense_mediated_decay", "lncRNA", "retained_intron", "cds_not_defined", "processed_transcript", "unnanotated (newly discovered"]), "transcript_biotype"] = "other"

## Drop gene_biotype column
final_ref.drop(columns="gene_biotype", inplace=True)

## Only keep CDS regions for protein_coding transcripts
final_ref_cds = final_ref.loc[final_ref["type"] == "CDS"].copy()
final_ref_cds_prot = final_ref_cds.loc[final_ref_cds["transcript_biotype"] == "protein_coding"].copy()
final_ref_2 = final_ref.loc[final_ref["type"] != "CDS"].copy()
final_final_ref = pd.concat([final_ref_2, final_ref_cds_prot]).sort_values(by=["chr", "start", "type", "end"], 
                                                                                    ascending=[True, True, False, True])

In [12]:
## Save files
final_final_ref.to_csv("../../../../data/bernardo/processed/99.other/create_annotations/annotation_for_maddy_r_shiny_app/annotation_r_shiny.tsv", sep="\t", 
                index=False)

In [13]:
## Fix expression matrix
df = fix_column_names(df, is_gene=False)
df = calculate_cpm(df, is_gene=False)

In [14]:
## Save expression Matrix
df.to_csv("../../../../data/bernardo/processed/99.other/create_annotations/annotation_for_maddy_r_shiny_app/expression_matrix_r_shiny.tsv", sep="\t",
         index=False)

# Create gene level file with total counts and CPM

In [15]:
## Import gene level bambu counts matrix
df_gene =  pd.read_csv("../../../../data/bernardo/raw/uky_aged_stringent/bambu_discovery/counts_gene.txt", 
                           delimiter="\t", low_memory=False, header=0)

In [16]:
## Calculate total_counts, drop other counts columns
df_gene = fix_column_names(df_gene, is_gene=True)
counts_columns = df_gene.columns[1:].to_list()
df_gene["total_counts"] = df_gene[counts_columns].sum(axis=1)
df_gene.drop(columns=counts_columns, inplace=True)

In [17]:
## Calculate CPM and give it proper name
df_gene = calculate_cpm(df_gene, is_gene=True)
df_gene["average_CPM"] = df_gene["total_CPM"].copy()
df_gene.drop(columns="total_CPM", inplace=True)

In [18]:
## Save file
df_gene.to_csv("../../../../data/bernardo/processed/99.other/create_annotations/annotation_for_maddy_r_shiny_app/expression_matrix_r_shiny_GENE.tsv",
              sep="\t", index=False)

# Create annotations to check for protein sequences for Maddy

In [21]:
## Open original reference
original_ref = pd.read_csv("../../../../references/bernardo/Homo_sapiens.GRCh38.107_ERCC.gtf", header=None, delimiter="\t", low_memory=False, 
                       names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"], comment="#")



## Bambu reference with novel and annotated transcripts
bambu_ref = pd.read_csv("../../../../data/bernardo/raw/uky_aged_stringent/bambu_discovery/extended_annotations.gtf", header=None, delimiter="\t",
                        low_memory=False, names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])


In [22]:
## Filter only new transcripts
bambu_ref = bambu_ref.loc[bambu_ref["other"].str.contains('transcript_id "BambuTx')]


## Filter only protein coding transcripts
original_ref = original_ref.loc[original_ref["other"].str.contains('transcript_biotype "protein_coding";')]



In [23]:
## Save files
bambu_ref.to_csv("../../../../data/bernardo/processed/99.other/create_annotations/annotation_for_maddy_r_shiny_app/only_new_transcripts_no_filter.gtf", sep="\t", 
                index=False, header=False, quoting=csv.QUOTE_NONE)

original_ref.to_csv("../../../../data/bernardo/processed/99.other/create_annotations/annotation_for_maddy_r_shiny_app/only_protein_coding_transcripts_no_filter.gtf", sep="\t", 
                index=False, header=False, quoting=csv.QUOTE_NONE)